In [2]:
# Install keras-tuner
!pip install keras-tuner --upgrade

# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
import numpy as np


# Load dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

# Split train into train and validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

# Define model builder for hyperparameter tuning
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(784,)))

    # Tune number of layers and units
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=64, max_value=512, step=64),
            activation='relu',
            kernel_regularizer=regularizers.l2(hp.Choice("l2_reg", [0.01, 0.001, 0.0001]))
        ))
        model.add(layers.Dropout(hp.Float("dropout", 0.2, 0.5, step=0.1)))

    model.add(layers.Dense(10, activation='softmax'))

    # Tune optimizer and learning rate
    optimizer = hp.Choice("optimizer", ["adam", "sgd"])
    lr = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])

    if optimizer == "adam":
        opt = keras.optimizers.Adam(learning_rate=lr)
    else:
        opt = keras.optimizers.SGD(learning_rate=lr)

    model.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Hyperparameter tuning using RandomSearch
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=1,
    directory="tuner_dir",
    project_name="mnist_dnn_tuning"
)

tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

# Get best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate on test data
test_loss, test_acc = best_model.evaluate(x_test, y_test)

print(f"\nTest Accuracy: {test_acc:.4f}")


Trial 5 Complete [00h 01m 31s]
val_accuracy: 0.9254999756813049

Best val_accuracy So Far: 0.9715833067893982
Total elapsed time: 00h 11m 21s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9668 - loss: 0.1480

Test Accuracy: 0.9710
